<p style="text-align:center">
    <a href="https://www.ict.mahidol.ac.th/en/" target="_blank">
    <img src="https://www3.ict.mahidol.ac.th/ICTSurveysV2/Content/image/MUICT2.png" width="400" alt="Faculty of ICT">
    </a>
</p>


# Lab10: Introduction to Time Series Modeling - Tutorial

Time Series modeling is an supervised learning technique for forecasting (generating a future outcome) by using historical data.


In [ ]:
from darts import TimeSeries
from darts.metrics import (
    coefficient_of_variation,
    mae,
    mape,
    #marre,
    mase,
    mse,
    rmse,
    ope,
    r2_score
)

# Part 1 - Univariate Forecasting: - ARIMA - Forecast a future point in time.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from darts.datasets import AirPassengersDataset

y = AirPassengersDataset().load().to_dataframe()
plt.plot(y)

## Task 1: Forecasting with Seasonal-Trend decomposition using LOESS (STL):
- https://www.statsmodels.org/stable/examples/notebooks/generated/stl_decomposition.html

In [ ]:
from statsmodels.tsa.seasonal import STL

stl = STL(y, seasonal=13)
res = stl.fit()
fig = res.plot()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.forecasting.stl import STLForecast

stlf = STLForecast(y[:-24], ARIMA, model_kwargs=dict(order=(1, 1, 0), trend="t"))
stlf_res = stlf.fit()
forecast = stlf_res.forecast(24)

plt.plot(y[:-24], label='train')
plt.plot(forecast, label='forecast')
plt.plot(y[-24:], label='val')
plt.legend()
plt.show()

In [ ]:
from darts.metrics import r2_score

r2_score(TimeSeries.from_values(y[-24:].to_numpy()), TimeSeries.from_values(forecast))

## Task 2: Forecasting with Exponential Smoothing:

Holt’s Winters Seasonal Exponential Smoothing including a trend component and a seasonal component.

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

model = ExponentialSmoothing(
    y[:-24],
    # seasonal_periods=60,
    trend="mul", # mul or add
    seasonal="add", # mul or add
    use_boxcox=True,
    initialization_method="estimated",
).fit()

forecast = model.forecast(24) 


plt.plot(y[:-24], label='train')
plt.plot(forecast, label='forecast')
plt.plot(y[-24:], label='val')
plt.legend()
plt.show()

r2_score(TimeSeries.from_values(y[-24:].to_numpy()), TimeSeries.from_values(forecast))

## Task 3: Forecasting with LSTM:

In [ ]:
series = TimeSeries.from_dataframe(y)

# Create training and validation sets:
train, val = series[:-24], series[-24:]

In [ ]:
val.start_time()

Scale Transform:

In [ ]:
from darts.dataprocessing.transformers import Scaler


# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

Static Covariates:
- Static covariates are characteristics of a time series / constants which do not change over time. When dealing with multiple time series, static covariates can help specific models improve forecasts. Darts' models will only consider static covariates embedded in the target series (the series for which we want to predict future values) and not past and/or future covariates (external data).
- https://unit8co.github.io/darts/examples/15-static-covariates.html

In [ ]:
import pandas as pd
from darts.utils.timeseries_generation import datetime_attribute_timeseries

# create month and year covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(), freq=series.freq_str, periods=200),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)

In [ ]:
print('Standard Scaled Years:')
display(year_series.to_dataframe())
print('One hot encoded Months:')
display(month_series.to_dataframe())

Add these new features as "covariates" (extra features) to the model, along with the target `Time Series data`:
- Month as one-hot encoded features.
- Year as numeric feature, standard scaled.

In [ ]:
covariates = year_series.stack(month_series)
cov_train, cov_val = covariates.split_after(val.start_time())

In [ ]:
from darts.models import RNNModel

model = RNNModel(
    model="LSTM",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=50,
    optimizer_kwargs={"lr": 1e-3},
    model_name="Air_RNN",
    log_tensorboard=True,
    random_state=42,
    training_length=20,
    input_chunk_length=14,
    force_reset=True,
    save_checkpoints=True,
)

model = model.fit(
    train_transformed,
    future_covariates=covariates,
    val_series=val_transformed,
    val_future_covariates=covariates,
    verbose=True,
)

In [ ]:
# best_model = RNNModel.load_from_checkpoint(model_name="Air_RNN", best=True)
forecast = model.predict(n=24, future_covariates=covariates)
print(forecast.shape)

plt.plot(train_transformed.to_dataframe(), label='train')
plt.plot(forecast.to_dataframe(), label='forecast')
plt.plot(val_transformed.to_dataframe(), label='val')
plt.legend()
plt.show()

r2_score(val_transformed, forecast)

- Try changing Epochs to 20, 50 and more to observe the changes in R2_Scores.

```












```
# Part 2 - Data Exploration Steps for Time Series Models:
- Largely these steps are required by Statistical models depending on `Autoregression` (ARIMA, VARIMA, VAR, etc).
- They can be informative to understand the conditions of the data's correlations and dependencies over time to guide `ML-based modelling` choices.

##### Dataset:
Let's use the airline passengers dataset to illustrate these steps:

In [ ]:
from darts.datasets import AirPassengersDataset

y = AirPassengersDataset().load().to_dataframe()
plt.plot(y)

## Task 1 - Stationarity - Testing & Transforms:

- Test using Augmented Dickey-Fuller (ADF) inference test to confirm significant trend (`stationarity`), if so let's remove.
- *Strictly necessary for autoregressive models: such as ARIMA, VARIMA, VAR.*

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(y)
print("ADF Statistic:", result[0])
print("p-value:", result[1])

##### Non-Stationary to Stationary Transforms (`statsmodels -> detrend`)
- https://www.statsmodels.org/stable/generated/statsmodels.tsa.tsatools.detrend.html

In [ ]:
from statsmodels.tsa.tsatools import detrend
y_detrend = detrend(y, order=3, axis=0)
plt.plot(y)
plt.plot(y_detrend)

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(y_detrend)
print("ADF Statistic:", result[0])
print("p-value:", result[1])

- With a P-value below 0.05, we can consider the test result is significant. Now the data is "stationary".

##### Non-Stationary to Stationary Transforms (`diff(y)`) - First order difference

In [ ]:
y_pd = y
y_diff = y_pd.diff()
plt.plot(y_pd, label='Y')
plt.plot(y_diff, label='diff(Y)')
plt.legend()

##### Non-Stationary to Stationary Transforms (`diff(diff(Y))`) - Second Order Difference

In [ ]:
y_pd = y
y_diff = y_pd.diff().diff()
# plt.plot(y_pd)
plt.plot(y_pd, label='Y')
plt.plot(y_diff, label='diff(diff(Y))')
plt.legend()

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(y_diff.dropna())
print("ADF Statistic:", result[0])
print("p-value:", result[1])

##### Non-Stationary to Stationary Transforms (`diff(Log(y))`) - Log and Difference

In [ ]:
y_pd = y
y_log = np.log(y_pd)
y_log_diff = y_log.diff()
# plt.plot(y_pd)
plt.plot(y_log, label='log(Y)')
plt.plot(y_log_diff, label='diff(log(Y))')
plt.legend()

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(y_log_diff.dropna())
print("ADF Statistic:", result[0])
print("p-value:", result[1])

##### Non-Stationary to Stationary Transforms (`helper function`)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

sd = seasonal_decompose(y)
_ = sd.plot()

In [ ]:
_ = plt.plot(seasonal_decompose(y_pd).resid)

In [ ]:
seasonal_decompose(y_pd).resid.dropna()

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(seasonal_decompose(y_pd).resid.dropna())
print("ADF Statistic:", result[0])
print("p-value:", result[1])

In [ ]:
y_detrend = seasonal_decompose(y_pd).resid.dropna()

## Task 2 - Lag Features: Autocorrelation Factor (ACF) and Partial Autocorrelation Factor (PACF) Plots: 

> The value of PACF is to measure the correlational effect of lag on future predictions; which is the periodicity of cycles in the time series data. We look back at the historical data to predict the future data. A good choice here can improve the predictive performance of the your model.

#### Requirements:


##### Stationarity (As Above in Task 1):
- In order to correctly evaluate PACF, we should ensure the (univariate) series is `stationary`.
- Although not strictly required, later correlation-lags will be less sensitive. 
    - Evaluate whether the series is stationary (often not the case) using the ADF and KPSS tests.
    - Convert `Non-Stationary` (univariate) series to `Stationary` by using `diff()` (within pandas), which calculates the derivative of the sequence. Sometimes we need the first deriviative (e.g. at order 1, `diff() once`) and sometimes we need more, e.g. order 2, `diff() twice`; sometimes it's not possible to achieve stationarity.
- Once we have stationary time series data, we can correctly interpret the PACF.

##### Intepretting Lag Feature Effects:
- PACF expresses the `correlation` between a univariate variable (a time series, e.g. $y_t$) and its lagged variable (e.g. $y_{t-lag}$).
- PACF excludes any effects of the intermediary lagged time steps (i.e. $y_t-1$, $y_t-2$, $y_t-3$, ... to $y_{t-(lag-1)}$).
- `Correlation` between two variables indicates how much they change together, `a=[1,2,3]` with `b=[1,2,2.9]` is highly correlated for example, and a correlation test (such as Pearson's R) would calculate a high correlation score.
- In PACF, we consider the true data e.g. `a=[10,20,30]` and lagged version (e.g. `lag=1`) of that data `b=[--,10,20]`. In this case the correlation is quite high (because the example is a linear line :-).
- If we find a lag value (e.g. lag=1,2,3, etc.) that gives a `high correlation score` it indicates that this lag value is a good one (is helpful for predicting the true values).

##### How to read the PACF plot:
- `Y-axis` shows `correlation` score.
- `X-axis` shows each `Lag` value, starting at 0 (only the original data) and the number you ask for (e.g. 5).
- The `grey` area that starts from the left at `Lag=1` determines the `confidence intervals (CI)` (default is alpha=0.05, which is CI=95%).
    - Btw, In PACF, the `grey area increases in size` as more comparisons that are made (from left to right). This is a probability correction activity (like a Bonferroni Correction for Family-wise Error); it reduces the random chance of the test having made  mistake (error), or us as the plot reader.
- Any `Lag`-`Correlation` points that fall within that `grey` area are `insignificant` (statistically), and we can forget about that particular lag value as being important (for now).
- The Granger Causality test (`statsmodels.tsa.stattools.grangercausalitytests`) will evaluate whether a single lagged series can predict the true series (via statistical significance testing).
- Each of the lagged series (lag values) showing as significant (by Granger Test or by exceeding the 95% Confidence Intervals in the plot) can be used as predictors of the true series.

> PACF is therefore a form of (independent-feature) feature selection, in order to help improve time series model predictions.

In [ ]:
plt.plot(y)
_ = plt.title('Airline Passenger Dataset:')

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

lags = [24, 60]
for lag in lags:
    print('Lag Parameter is: ',lag)
    fig = plot_pacf( y_detrend , lags=lag, alpha=0.05, title='Partial Autocorrelation Factor (PACF)') 
    axs = fig.get_axes()
    [ax.set_ylabel('Correlation Score') for ax in axs]
    [ax.set_xlabel('Lag Value') for ax in axs]
    plt.show()

### Reflection of PACF results:
- Lag-independent negative correlations at months 2,3,6,7,8,13: - indicates decreased passengers on these months.
- Lag-independent positive correlations at months 11,12: - indicates increased passengers on these months.
- Lag-independent positive correlations at month 1: - indicates increasing trend of passengers.

### Autocorrelation Factor (ACF) Plots:
- Autocorrelation Factor illustrates the dependent (correlated) relationship between each lag and its previous lags.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

lags = [24, 60]
for lag in lags:
    print('Lag Parameter is: ',lag)
    fig = plot_acf( y_detrend , lags=lag, alpha=0.05, fft=True, title='Autocorrelation Factor (ACF)') 
    axs = fig.get_axes()
    [ax.set_ylabel('Correlation Score') for ax in axs]
    [ax.set_xlabel('Lag Value') for ax in axs]
    plt.show()

### Reflection of ACF results:
- A significant and positive (increasing) correlation at cycle periods of 11,12,13 months and at 1 month.
    - Indicating at 1 month, trending upwards.
    - Indicating at 11,12,13 months treading upwards on a yearly (11 to 13 month) basis: - Suggesting 3 months of the year are regularly high and trending upwards.
- A significant and negative (decreasing) correlation at cycle periods of 4,5,6,7,8 months: - 5 months of the year are regularly low and have lessening correlation over time.
- Overall, time-dependency correlations (and trends) are significant in this dataset.

```









```

```






```
<p style="text-align:center;">That's it! Congratulations! <br>     
    Now, let's move to today's Lab Assignment.</p>